<a href="https://colab.research.google.com/github/auzaluis/upsa_mod_202502/blob/master/whisky/01_script_EDA_whisky.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import plotly.express as px
import re

In [ ]:
# Importar parquet
dir = '/content/drive/MyDrive/Teaching/UPSA/2025.02/modelacion/whisky/data-whisky.parquet'

In [ ]:
df = pd.read_parquet(dir)

Situación Competitiva | Funnel de marca

Lealtad

In [ ]:
df_lealtad = (
    df['¿Cuál es la marca que más compra?']
    .value_counts()
    .reset_index()
    .rename(columns={
        '¿Cuál es la marca que más compra?': 'Marca',
        'count': 'n'
    })
)

In [ ]:
df_lealtad['Porcentaje'] = df_lealtad['n'] / df_lealtad['n'].sum()
df_lealtad['Porcentaje_txt'] = df_lealtad['Porcentaje'].apply(lambda x: str(round( x*100, 0)) + '%')
df_lealtad

In [ ]:
px.bar(
    df_lealtad,
    x='Marca',
    y='Porcentaje',
    text='Porcentaje_txt',
    title='Lealtad: ¿Cuál es la marca que más compra?'
).update_layout(
    template='plotly_white',
    yaxis_showticklabels=False,
    yaxis_showgrid=False,
    yaxis_title='',
    xaxis_title=''
)

Conocimiento

In [ ]:
df.columns

In [ ]:
[col for col in df.columns if col.startswith('Conocimiento')]

In [ ]:
df_conocimiento_long = df.melt(
    id_vars=[],
    value_vars=[col for col in df.columns if col.startswith('Conocimiento')],
    var_name='KPI',
    value_name='Marca'
).dropna()

In [ ]:
df_conocimiento_long.head()

In [ ]:
df_conocimiento = (
    df_conocimiento_long['Marca']
    .value_counts()
    .reset_index()
    .rename(columns={'count': 'n'})
)

In [ ]:
df_conocimiento['Porcentaje'] = df_conocimiento['n'] / len(df)
df_conocimiento['Porcentaje_txt'] = df_conocimiento['Porcentaje'].apply(lambda x: str(round( x*100, 0)) + '%')
df_conocimiento

In [ ]:
px.bar(
    df_conocimiento,
    x='Marca',
    y='Porcentaje',
    text='Porcentaje_txt',
    title='Conocimiento: Cuáles de las siguiente marcas conoce?'
).update_layout(
    template='plotly_white',
    yaxis_showticklabels=False,
    yaxis_showgrid=False,
    yaxis_title='',
    xaxis_title=''
)

Prueba

In [ ]:
df_prueba_long = df.melt(
    id_vars=[],
    value_vars=[col for col in df.columns if col.startswith('Prueba')],
    var_name='KPI',
    value_name='Marca'
).dropna()

In [ ]:
df_prueba = (
    df_prueba_long['Marca']
    .value_counts()
    .reset_index()
    .rename(columns={'count': 'n'})
)

In [ ]:
df_prueba['Porcentaje'] = df_prueba['n'] / len(df)
df_prueba['Porcentaje_txt'] = df_prueba['Porcentaje'].apply(lambda x: str(round( x*100, 0)) + '%')

Consideración

In [ ]:
df['Consideración Chivas'].unique()

In [ ]:
consideracion_t2b = ['Creo que SÍ la consideraría', 'Sería mi 1ra opción']
consideracion_t2b

In [ ]:
# Ejercicio suelto: Aplicación del top2box
df['Consideración Johnny Walker'].apply(lambda x: x in consideracion_t2b).sum()

In [ ]:
# Ejercicio suelto: Extracción de la marca del nombre de la columna
re.search(r'(Consideración\s)(.+)', 'Consideración Chivas').group(2)

In [ ]:
# Columnas que inician con: Consideración
consideracion_cols = [col for col in df.columns if col.startswith('Consideración')]
consideracion_cols

In [ ]:
consideracion_dict = []

for col in consideracion_cols:
  Total = len(df)
  n = df[col].apply(lambda x: x in consideracion_t2b).sum()
  Porcentaje = n / Total
  Porcentaje_txt = str(round(Porcentaje*100, 0)) + '%'
  Marca = re.search(r'(Consideración\s)(.+)', col).group(2)
  consideracion_dict.append({
      'Marca': Marca,
      'n': n,
      'Porcentaje': Porcentaje,
      'Porcentaje_txt': Porcentaje_txt
  })


In [ ]:
df_consideracion = pd.DataFrame(consideracion_dict)
df_consideracion

Recomendación

In [ ]:
recomendacion_t2b = [9,10]
recomendacion_t2b

In [ ]:
recomendacion_cols = [col for col in df.columns if col.startswith('Recomendación')]
recomendacion_cols

In [ ]:
recomendacion_dict = []

for col in recomendacion_cols:
  Total = len(df)
  n = df[col].apply(lambda x: x in recomendacion_t2b).sum()
  Porcentaje = n / Total
  Porcentaje_txt = str(round(Porcentaje*100, 0)) + '%'
  Marca = re.search(r'(Recomendación\s)(.+)', col).group(2)
  recomendacion_dict.append({
      'Marca': Marca,
      'n': n,
      'Porcentaje': Porcentaje,
      'Porcentaje_txt': Porcentaje_txt
  })

In [ ]:
df_recomendacion = pd.DataFrame(recomendacion_dict)
df_recomendacion

In [ ]:
df_conocimiento['KPI'] = 'Conocimiento'
df_consideracion['KPI'] = 'Consideración'
df_prueba['KPI'] = 'Prueba'
df_lealtad['KPI'] = 'Lealtad'
df_recomendacion['KPI'] = 'Recomendación'

In [ ]:
df_funnel = pd.concat([
    df_conocimiento,
    df_consideracion,
    df_prueba,
    df_lealtad,
    df_recomendacion
])

In [ ]:
df_funnel['Marca'] = df_funnel['Marca'].apply(lambda x: 'Chivas Regal' if x == 'Chivas' else x)
df_funnel = df_funnel[df_funnel['Marca'] != 'Ninguno']

In [ ]:
px.bar(
    df_funnel,
    x = 'Marca',
    y = 'Porcentaje',
    color = 'KPI',
    text = 'Porcentaje_txt',
    barmode = 'group',
    title = 'Funnel de Marcas de Whisky',
    color_discrete_map = {
        'Conocimiento': '#72efdd',
        'Consideración': '#56cfe1',
        'Prueba': '#4ea8de',
        'Lealtad': '#5e60ce',
        'Recomendación': '#6930c3'
        }
).update_layout(
    template = 'plotly_white',
    yaxis_showticklabels = False,
    yaxis_ticks = '',
    xaxis_ticks = '',
    xaxis_title = None,
    yaxis_title = None,
    legend = {'orientation': 'h', 'y': 1.1},
    yaxis_showgrid = False
).update_traces(
    textfont_size = 10,
    textangle = 0
)